# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value

print(sum)


8.382332347441762


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_1():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}


In [4]:

# Question 2: What is the 13th number yielded
limit = 13
generator = square_root_generator(limit)

for i, sqrt_value in enumerate(generator):
  if i == limit - 1:
    print(f"The 13th number : {sqrt_value}")

The 13th number : 3.605551275463989


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**



# Solution: First make sure that the following modules are installed:

In [5]:
#Install the dependencies
! pip install dlt[duckdb]

zsh:1: no matches found: dlt[duckdb]


In [6]:
import dlt

/Users/sarathchandrika/Documents/Multi-News/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
import duckdb

In [8]:
import pandas as pd 
import numpy as np

In [13]:
data1 = []
for person in people_1():
  data1.append(person)

data2 = []
for person in people_2():
  data2.append(person)

# define the connection to load to.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

# run with append write disposition.
info = pipeline.run(data1,
										table_name="people1",
										write_disposition="append")
# show the outcome
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.21 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/sarathchandrika/Downloads/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708485283.2938392 is LOADED and contains no failed jobs


In [15]:

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# let's see the tables
print('Loaded tables: ')
display(conn.sql("show tables"))

people_table = conn.sql("SELECT * FROM people1").df()
display(people_table)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people1             │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708485283.2938392,p9L5N088zBjGNw
1,2,Person_2,27,City_A,1708485283.2938392,Nqooodhigx+pkw
2,3,Person_3,28,City_A,1708485283.2938392,BO2xv2sYVHqajQ
3,4,Person_4,29,City_A,1708485283.2938392,bYZEZmiq6B5PTA
4,5,Person_5,30,City_A,1708485283.2938392,Hr1GJnYraW/9XQ


In [16]:
query = conn.sql("SELECT SUM(age) AS total_ages FROM people1").df()
display(query)

,total_ages
0,140.0


In [17]:
info = pipeline.run(data2, table_name='people1', write_disposition="append")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/sarathchandrika/Downloads/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708485324.550653 is LOADED and contains no failed jobs


In [46]:
people_table = conn.sql("SELECT * FROM people").df()
display(people_table)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708484656.751055,b08XZboKzXI/qA,None
1,2,Person_2,27,City_A,1708484656.751055,MrdaBqnjobefzg,None
2,3,Person_3,28,City_A,1708484656.751055,NxS0c8Ts1s9VVA,None
3,4,Person_4,29,City_A,1708484656.751055,BKzOxrRcMd3hLw,None
4,5,Person_5,30,City_A,1708484656.751055,sbRlSx2U8t3vKg,None
5,1,Person_1,26,City_A,1708484802.670677,th/VgVTb20xvMw,None
6,2,Person_2,27,City_A,1708484802.670677,wwZ0OuS5Af6LbA,None
7,3,Person_3,28,City_A,1708484802.670677,tpHBCxV4pJBO7Q,None
8,4,Person_4,29,City_A,1708484802.670677,z2j6i8y317vu7Q,None
9,5,Person_5,30,City_A,1708484802.670677,P8G3EcOeGEotHw,None


In [18]:
query = conn.sql("SELECT SUM(age) AS total_ages FROM people1").df()
display(query)

,total_ages
0,353.0


In [19]:

# run with append write disposition.
info = pipeline.run(data1, table_name="people_merge", write_disposition="append")
# show the outcome
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.22 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/sarathchandrika/Downloads/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708485408.687253 is LOADED and contains no failed jobs


In [20]:
# run with append write disposition.
info = pipeline.run(data2, table_name="people_merge", write_disposition="merge", primary_key="id")
# show the outcome
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/sarathchandrika/Downloads/dlt_ipykernel_launcher.duckdb location to store data
Load package 1708485438.271855 is LOADED and contains no failed jobs


In [21]:
people_merge_table = conn.sql("SELECT * FROM people_merge").df()
display(people_merge_table)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708485408.687253,17ycgOMJGujvtg,None
1,2,Person_2,27,City_A,1708485408.687253,K5hPP7LT67RgIw,None
2,4,Person_4,34,City_B,1708485438.271855,I0ouwS6DvpIGLw,Job_4
3,5,Person_5,35,City_B,1708485438.271855,j+aTocUAVc1LPA,Job_5
4,3,Person_3,33,City_B,1708485438.271855,d0ZojrP6N7kkaw,Job_3
5,6,Person_6,36,City_B,1708485438.271855,BQtfKnSP2lT9lw,Job_6
6,8,Person_8,38,City_B,1708485438.271855,9WVHZx5fZosoKA,Job_8
7,7,Person_7,37,City_B,1708485438.271855,3n/bjhpotNIX8g,Job_7


In [22]:
query = conn.sql("SELECT SUM(age) AS total_ages FROM people_merge").df()
display(query)

,total_ages
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX